<h1 align="center">EQE512 MATRIX METHODS IN STRUCTURAL ANALYSIS 
<br>
<br>
Week 06
<br>
<br>
Beam member w/ span load & Column member &  Frame Systems</h1> 

<h3 align="center">Dr. Ahmet Anıl Dindar (adindar@gtu.edu.tr)</h3> 
<h4 align="center">2021 Fall  </h4> 

---
**Today's Topics**

1- Example with beam

2- Column member stiffness matrix

3- Frame System stiffness matrix

---

# 1- Loaded Beam

**For the given system, draw the internal force diagrams. (Material C25)**

<img src="./figures/6-Example-Beam_loaded.png" width = "600">

**Numbering and mapping of the system displacments and forces.**

<img src="./figures/6-Example-Beam_loaded-2.png" width="600">

---
**Solution :**

1. Obtain the equivalent end forces
2. Apply the obtained values in opposite directions at the nodes
3. Ignore the span loads

<img src="./figures/12-EQE512-Loads-Apply.PNG"   style="width:70%">

---

In [1]:
# Mapping and numbering
DOF = 2
#Nodes' names with coordinates
joint_nodes = {1:[0,0] , 2:[0,4], 3:[0,7]}

# Members' names and nodes
member_nodes= {1:[1,2] , 2:[2,3]}

# Member properties
member_1 = {"E":30_000_000 ,  "I" : round((.250 * .500**3 )/12 ,10) ,  
                "jointI": joint_nodes[1] , 
                "jointJ": joint_nodes[2]}

member_2 = {"E":30_000_000 ,  "I" : round((.250*.500**3)/12 ,10) ,  
                "jointI": joint_nodes[2] , 
                "jointJ": joint_nodes[3]}

member_props = {}

for member_no , member_property in zip( member_nodes.keys() , [member_1 ,  member_2] ):
    
    member_props[ member_no ] = member_property   
    
member_props

{1: {'E': 30000000, 'I': 0.0026041667, 'jointI': [0, 0], 'jointJ': [0, 4]},
 2: {'E': 30000000, 'I': 0.0026041667, 'jointI': [0, 4], 'jointJ': [0, 7]}}

**Fixed End Moment**

In [2]:
def beam_load_single_force(f,a,b):
    L = a+b
    s1 = f*( 1 - (3*a**2)/(L**2) + (2*a**3)/(L**3) )
    s2 = f*( a - (2*a**2)/(L) + (a**3)/(L**2) )
    s3 = f*( (3*a**2)/(L**2) - (2*a**3)/(L**3) )
    s4 = f*( -1 * (a**2)/(L) + (a**3)/(L**2) )
    
    return(s1,s2,s3,s4)

In [3]:
def beam_load_uniform_force(f,L):
    s1 = (f * L) / 2
    s2 = (f * L**2) / 12
    s3 = (f * L) / 2
    s4 = (-1 * f * L**2) / 12    
    
    return(s1,s2,s3,s4)

In [4]:
# For the member 1
s1_1,s1_2,s1_3,s1_4 = beam_load_single_force( 100 , 2,2)
print( s1_1,s1_2,s1_3,s1_4 )

50.0 50.0 50.0 -50.0


In [5]:
# For the member 2
s2_1,s2_2,s2_3,s2_4 = beam_load_uniform_force( 20 , 3)
print( s2_1,s2_2,s2_3,s2_4 )

30.0 15.0 30.0 -15.0


In [6]:
# Load Vector - Manuelly created P vector
# Can you make its algorithm?

In [7]:
import numpy as np 

P = np.array([-50 , -50 , -80 , 35, -30 , 15])
print( P )

[-50 -50 -80  35 -30  15]


<img src="./figures/6-Example-Beam_loaded-3.png" width="600">

**Import the previous course notebook**

In [8]:
import EQE512_functions

In [9]:
# Data structure for member stiffness matrices
k_members = {}

for member_no , member_property in member_props.items() : 
    print( member_no)
    k_members[member_no] =  EQE512_functions.beam_stiffness_matrix(member_property  )
    
    print( "="*100 , "\n" , k_members[member_no])
    
print( "~"*100)  

1
4.0
 [[ 14648.4376875  29296.875375  -14648.4376875  29296.875375 ]
 [ 29296.875375   78125.001     -29296.875375   39062.5005   ]
 [-14648.4376875 -29296.875375   14648.4376875 -29296.875375 ]
 [ 29296.875375   39062.5005    -29296.875375   78125.001    ]]
2
3.0
 [[ 34722.22266667  52083.334      -34722.22266667  52083.334     ]
 [ 52083.334      104166.668      -52083.334       52083.334     ]
 [-34722.22266667 -52083.334       34722.22266667 -52083.334     ]
 [ 52083.334       52083.334      -52083.334      104166.668     ]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [22]:
K_system = EQE512_functions.system_stiffness_matrix( k_members , joint_nodes , member_nodes , DOF = 2)
print( K_system)

[[ 14648  29296 -14648  29296      0      0]
 [ 29296  78125 -29296  39062      0      0]
 [-14648 -29296  49370  22787 -34722  52083]
 [ 29296  39062  22787 182291 -52083  52083]
 [     0      0 -34722 -52083  34722 -52083]
 [     0      0  52083  52083 -52083 104166]]


**Apply Boundary Conditions**

In [12]:
P_factor = np.array( [ 0 , 0 , 0, 1 , 0 , 1 ] )

In [13]:
U_factor = np.array( [ 0 , 0 , 0, 1 , 0 , 1 ] )

In [14]:
K_bc = EQE512_functions.apply_boundary_conditions( K_system , U_factor , P_factor)
K_bc

array([[     1,      0,      0,      0,      0,      0],
       [     0,      1,      0,      0,      0,      0],
       [     0,      0,      1,      0,      0,      0],
       [     0,      0,      0, 182291,      0,  52083],
       [     0,      0,      0,      0,      1,      0],
       [     0,      0,      0,  52083,      0, 104166]])

In [15]:
P_bc = P*P_factor
P_bc

array([ 0,  0,  0, 35,  0, 15])

** Solve the relation between Force and Displacement**

$  K_{bc} \times U = P_{bc}$

In [20]:
U = np.matmul( np.linalg.inv( K_bc ) , P_bc) 

[print(f"{item} m or rad") for item in U];

0.0 m or rad
0.0 m or rad
0.0 m or rad
0.00017600056320180225 m or rad
0.0 m or rad
5.600064000499715e-05 m or rad


**Solution**

$  K \times U = P$  

In [25]:
P_solution = np.matmul( K_system , U)
[print(f"{round(item,2)} kN or kNm") for item in P_solution];

5.16 kN or kNm
6.87 kN or kNm
6.93 kN or kNm
35.0 kN or kNm
-12.08 kN or kNm
15.0 kN or kNm


In [27]:
P_support_reactions = P_solution - P
[print(f"{round(item,2)} kN or kNm") for item in P_support_reactions];

55.16 kN or kNm
56.87 kN or kNm
86.93 kN or kNm
0.0 kN or kNm
17.92 kN or kNm
0.0 kN or kNm


---
## Inclass Example

**For the given system, draw the internal force diagrams. (Material C25)**

- Take the concentrated load is 1m far from the wall on the left. Obtain the support reactions.
- Compare your results with the one given above. 

<img src="./figures/6-Example-Beam_loaded.png" width = "600">

---

# Columns

**Freedom**

<img src="figures/9-EQE512-ColumnMemberExplanation.PNG" width="800">

---

**Transformation Matrix**

Remember the relation between local and global terms

$ \Large k^i = (T^i) \hat{k}^i T^i $ 

Remember the transformation matrix;

$ \Large T^i = \begin{bmatrix} \frac{\Delta_1}{L} & \frac{\Delta_2}{L}  & 0 & 0 & 0 & 0 \\ 
                              -\frac{\Delta_2}{L} & \frac{\Delta_1}{L}  & 0  & 0 & 0 & 0 \\ 
                               0 & 0 & 1 & 0 & 0 & 0  \\  
                               0 & 0 & 0 &  \frac{\Delta_1}{L} & \frac{\Delta_2}{L} 0 & 0 &   \\ 
                              0  & 0 & 0 & -\frac{\Delta_2}{L} & \frac{\Delta_1}{L} 0 & 0 &   \\ 
                               0 & 0 & 0 & 0 & 0 & 1  \\  
                               \end{bmatrix}$

---

**Local Stiffness Matrix**

<img src="figures/column_stiffness_in_local_coordinates.png" width="800">

**Global Stiffness Matrix**

After applying the transformation matrix

$ \Large k^i = (T^i)^T \hat{k}^i T^i$

Than the stiffness matrix in global terms

<img src="figures/9-EQE512-ColumnMemberStiffness.PNG" width="1000">

**Transformation Matrix of Frame Element**

In [ ]:
def transformation_matrix( x1,y1,x2,y2) :
    delta_1 = (x1-x2)
    delta_2 = (y1-y2)
    L = ( (delta_1)**2 + (delta_2)**2)**(0.5)
    
    T = np.array( [ ???????? ])
    
    return( T)

**Frame Element Member Stiffness Matrix in Local Coordinates**

In [ ]:
def column_stiffness_matrix():
    ?????????????
    

**Frame Element Member Stiffness Matrix in Global Coordinates**

In [ ]:
K_global =  ????????

---

**NEXT WEEK**
The topics for the next week 

- "Frame Systems- MRF Support displacement"